In [66]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
from datetime import datetime
import seaborn as sns
import pickle
sns.set()
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split  
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import confusion_matrix, r2_score
from sklearn.linear_model import LinearRegression  
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn import preprocessing
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

In [67]:
df=pd.read_csv('LoanExport.csv')

In [68]:
# Changing  FirstPaymentDate and MaturityDate into date format
df['FirstPaymentDate'] = pd.to_datetime(df['FirstPaymentDate'], format='%Y%m')
df['MaturityDate'] = pd.to_datetime(df['MaturityDate'], format='%Y%m')

In [69]:
df.head()

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,1999-02-01,N,2029-01-01,16974,25,1,O,89,27,...,60400,F199Q1268030,P,360,2,FL,WASHINGTONMUTUALBANK,0,0,52
1,0,1999-02-01,N,2029-01-01,19740,0,1,O,73,17,...,80200,F199Q1015092,N,360,1,FT,CHASEHOMEFINANCELLC,0,0,144
2,0,1999-02-01,N,2029-01-01,29940,0,1,O,75,16,...,66000,F199Q1266886,N,360,2,FL,WASHINGTONMUTUALBANK,0,0,67
3,0,1999-02-01,N,2029-01-01,31084,0,1,O,76,14,...,90700,F199Q1178167,N,360,2,GM,GMACMTGECORP,0,0,35
4,0,1999-02-01,N,2029-01-01,35644,0,1,O,78,18,...,7600,F199Q1178517,N,360,2,GM,GMACMTGECORP,0,0,54


In [70]:
def calculate_monthly_income(dti, emi):
    dti = dti if dti <=1 else dti / 100
    # Calculate montly income
    if dti == 0:
        monthly_income = emi
    else:
        monthly_income = emi / dti
    return np.int32(monthly_income)

In [71]:
def calculateEmi(principal, monthly_interest_rate, loan_term_months):
    numerator = (1 + monthly_interest_rate) ** loan_term_months
    denominator = numerator - 1
    interest = numerator / denominator
    emi = principal * monthly_interest_rate * interest
    return np.int32(emi)

In [72]:
def get_currentUPB(principal, monthly_interest_rate, monthly_installment,
                   payments_made):
    monthly_interest = monthly_interest_rate * principal
    monthly_paid_principal = monthly_installment - monthly_interest
    unpaid_principal = principal - (monthly_paid_principal * payments_made)
    return np.int32(unpaid_principal)

In [73]:
def calculatePrepayment(dti, monthly_income):
    if (dti < 40):
        prepayment = monthly_income / 2;
    else:
        prepayment = monthly_income * 3 / 4;
    return np.int32(prepayment)

In [74]:
df.head()

,CreditScore,FirstPaymentDate,FirstTimeHomebuyer,MaturityDate,MSA,MIP,Units,Occupancy,OCLTV,DTI,...,PostalCode,LoanSeqNum,LoanPurpose,OrigLoanTerm,NumBorrowers,SellerName,ServicerName,EverDelinquent,MonthsDelinquent,MonthsInRepayment
0,0,1999-02-01,N,2029-01-01,16974,25,1,O,89,27,...,60400,F199Q1268030,P,360,2,FL,WASHINGTONMUTUALBANK,0,0,52
1,0,1999-02-01,N,2029-01-01,19740,0,1,O,73,17,...,80200,F199Q1015092,N,360,1,FT,CHASEHOMEFINANCELLC,0,0,144
2,0,1999-02-01,N,2029-01-01,29940,0,1,O,75,16,...,66000,F199Q1266886,N,360,2,FL,WASHINGTONMUTUALBANK,0,0,67
3,0,1999-02-01,N,2029-01-01,31084,0,1,O,76,14,...,90700,F199Q1178167,N,360,2,GM,GMACMTGECORP,0,0,35
4,0,1999-02-01,N,2029-01-01,35644,0,1,O,78,18,...,7600,F199Q1178517,N,360,2,GM,GMACMTGECORP,0,0,54


In [75]:
df.drop(['SellerName','ProductType','LoanSeqNum'],axis=1,inplace=True) ## Dropping the irrelevant features

In [76]:
for var in ['MSA','PropertyType','NumBorrowers','PPM']: ## Loop containing variables used for handling nan values
    df[var]=df[var].str.strip() ## Removing the white space using str.strip()
    df[var]=df[var].replace('X','Other') ## Replacing nan values with a category "Other"`

In [77]:
mode=df['FirstTimeHomebuyer'].mode()
df['FirstTimeHomebuyer']=df['FirstTimeHomebuyer'].fillna(mode)[0]

In [78]:
## Function for removing outliers with 85th percentile and 5th percentile
def remove_outliers(df2,variable):
    
    upper_limit=df[variable].quantile(0.85)
    lower_limit=df[variable].quantile(0.15)
    df[variable]=np.where(df[variable]<=upper_limit,upper_limit,
                         np.where(df[variable]>=lower_limit,lower_limit,
                                 df[variable])) ## We want values lower than upper limit and larger than lower limit

In [79]:
variables=['CreditScore','FirstPaymentDate','MaturityDate','LTV','OCLTV','OrigUPB','OrigInterestRate','OrigLoanTerm',
          'MonthsDelinquent','MonthsInRepayment']

for variable in variables:
    remove_outliers(df,variable) ## Loop to remove outliers for the variables in the list

In [80]:
for var in ['Occupancy','Channel','PPM','LoanPurpose','NumBorrowers','FirstTimeHomebuyer','PropertyState','PropertyType','PostalCode','ServicerName','MSA']:
    ordinal_mapping={k:i for i,k in enumerate(df[var].unique(),0)} ## Assigning labels starting from 0
    df[var]=df[var].map(ordinal_mapping) ## Encoding the variables with the labels assigned through the loop written above

In [81]:
df['OrigInterestRate_Monthly'] =  np.round((df['OrigInterestRate'] / 12) / 100, 4)

In [82]:
df['MonthlyInstallment'] = df.apply(
        lambda features: calculateEmi(
            principal=features['OrigUPB'], 
            monthly_interest_rate=features['OrigInterestRate_Monthly'],
            loan_term_months=features['OrigLoanTerm']), axis=1)

In [83]:
df['CurrentUPB'] = df.apply(
        lambda features: get_currentUPB(
            monthly_interest_rate=features['OrigInterestRate_Monthly'],
            principal=features['OrigUPB'], 
            monthly_installment=features['MonthlyInstallment'],
            payments_made=features['MonthsInRepayment']), axis=1)

In [84]:
df['MonthlyIncome'] = df.apply(
        lambda features: calculate_monthly_income(
            dti = features['DTI'],
            emi = features['MonthlyInstallment']), axis =1)

In [85]:
df['Prepayment'] = df.apply(
        lambda features: calculatePrepayment(
            dti=features['DTI'],
            monthly_income=features['MonthlyIncome']), axis=1)

In [86]:
df['Totalpayment'] = df['MonthlyInstallment'] * df['OrigLoanTerm']

In [87]:
df['InterestAmount'] = df['Totalpayment'] - df['OrigUPB']

In [88]:
related_columns = ['OrigUPB', 'OrigInterestRate_Monthly', 'OrigLoanTerm',
                   'DTI', 'MonthsInRepayment' , 'MonthlyInstallment',
                   'CurrentUPB', 'MonthlyIncome', 'Prepayment', 'Totalpayment',
                   'InterestAmount']

In [89]:
df[related_columns]

,OrigUPB,OrigInterestRate_Monthly,OrigLoanTerm,DTI,MonthsInRepayment,MonthlyInstallment,CurrentUPB,MonthlyIncome,Prepayment,Totalpayment,InterestAmount
0,187000.0,0.006,360.0,27,98.0,1269,172594,4700,2350,456840.0,269840.0
1,187000.0,0.006,360.0,17,27.0,1269,183031,7464,3732,456840.0,269840.0
2,187000.0,0.006,360.0,16,98.0,1269,172594,7931,3965,456840.0,269840.0
3,187000.0,0.006,360.0,14,98.0,1269,172594,9064,4532,456840.0,269840.0
4,187000.0,0.006,360.0,18,98.0,1269,172594,7050,3525,456840.0,269840.0
...,...,...,...,...,...,...,...,...,...,...,...
291446,187000.0,0.006,360.0,32,98.0,1269,172594,3965,1982,456840.0,269840.0
291447,70000.0,0.006,360.0,0,98.0,475,64610,475,237,171000.0,101000.0
291448,187000.0,0.006,360.0,31,98.0,1269,172594,4093,2046,456840.0,269840.0
291449,187000.0,0.006,360.0,54,98.0,1269,172594,2350,1762,456840.0,269840.0


In [90]:
df['ActualPayments'] = df['MonthlyInstallment'] * df['MonthsInRepayment']

In [91]:
df['Prepayments'] = df['Prepayment'] // df['MonthlyInstallment']

In [92]:
df['ScheduledPayments'] = df['MonthlyInstallment'] * (df['MonthsInRepayment'] - df['Prepayments'] + df['MonthsDelinquent'])

In [93]:
df['PPR'] = (df['ScheduledPayments'] - df['ActualPayments']) / df['CurrentUPB']

In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 291451 entries, 0 to 291450
Data columns (total 36 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   CreditScore               291451 non-null  float64       
 1   FirstPaymentDate          291451 non-null  datetime64[ns]
 2   FirstTimeHomebuyer        291451 non-null  int64         
 3   MaturityDate              291451 non-null  datetime64[ns]
 4   MSA                       291451 non-null  int64         
 5   MIP                       291451 non-null  int64         
 6   Units                     291451 non-null  int64         
 7   Occupancy                 291451 non-null  int64         
 8   OCLTV                     291451 non-null  float64       
 9   DTI                       291451 non-null  int64         
 10  OrigUPB                   291451 non-null  float64       
 11  LTV                       291451 non-null  float64       
 12  Or

In [97]:
df = df.drop(['FirstPaymentDate','FirstTimeHomebuyer','LoanPurpose','MaturityDate','MIP','Units','MSA','NumBorrowers','Occupancy','Channel','PPM','OCLTV','PropertyState','PropertyType','PostalCode','ServicerName','MonthsDelinquent'],axis = 1)

In [98]:
df.head()

,CreditScore,DTI,OrigUPB,LTV,OrigInterestRate,OrigLoanTerm,EverDelinquent,MonthsInRepayment,OrigInterestRate_Monthly,MonthlyInstallment,CurrentUPB,MonthlyIncome,Prepayment,Totalpayment,InterestAmount,ActualPayments,Prepayments,ScheduledPayments,PPR
0,766.0,27,187000.0,93.0,7.25,360.0,0,98.0,0.006,1269,172594,4700,2350,456840.0,269840.0,124362.0,1,124362.0,0.000000
1,766.0,17,187000.0,93.0,7.25,360.0,0,27.0,0.006,1269,183031,7464,3732,456840.0,269840.0,34263.0,2,32994.0,-0.006933
2,766.0,16,187000.0,93.0,7.25,360.0,0,98.0,0.006,1269,172594,7931,3965,456840.0,269840.0,124362.0,3,121824.0,-0.014705
3,766.0,14,187000.0,93.0,7.25,360.0,0,98.0,0.006,1269,172594,9064,4532,456840.0,269840.0,124362.0,3,121824.0,-0.014705
4,766.0,18,187000.0,93.0,7.25,360.0,0,98.0,0.006,1269,172594,7050,3525,456840.0,269840.0,124362.0,2,123093.0,-0.007353


In [99]:
#Dividing Data in test and train
X=df.drop(['EverDelinquent','PPR'],axis=1)
y=df[['EverDelinquent','PPR']]

X_train,X_test,y_train,y_test=train_test_split(X,y, test_size=0.2, random_state=56)

# Separating Target values for classifications and regression problems
y_class_train = y_train.iloc[:,0]
y_class_test = y_test.iloc[:,0]

y_reg_test = y_test.iloc[:,1:]
y_reg_train = y_train.iloc[:,1:]

In [125]:
print(df['CreditScore'].min())
print(df['CreditScore'].max())

653.0
766.0


In [100]:
X_train.columns

Index(['CreditScore', 'DTI', 'OrigUPB', 'LTV', 'OrigInterestRate',
       'OrigLoanTerm', 'MonthsInRepayment', 'OrigInterestRate_Monthly',
       'MonthlyInstallment', 'CurrentUPB', 'MonthlyIncome', 'Prepayment',
       'Totalpayment', 'InterestAmount', 'ActualPayments', 'Prepayments',
       'ScheduledPayments'],
      dtype='object')

In [101]:
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
smote= SMOTE()
scaler = MinMaxScaler()

In [102]:

classifier_Pipe = Pipeline([
    ('scaler',scaler),
    ('randomforest',RandomForestClassifier(class_weight="balanced", max_depth=9))
     ])

In [103]:
classifier_Pipe.fit(X_train, y_class_train)

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('randomforest',
                 RandomForestClassifier(class_weight='balanced', max_depth=9))])

In [104]:
pred_class = classifier_Pipe.predict(X_test)
print('test accuracy = ', round(accuracy_score(y_class_test, pred_class)*100, 2), '%')

test accuracy =  90.02 %


In [105]:
print("Classification Report:")

print("\nAccuracy score:\n", round(accuracy_score(y_class_test, pred_class)*100,2), '%')
print('*'*40)
print("\nConfusion Matrix:\n", confusion_matrix(y_class_test, pred_class))
print('*'*40)
print("\nClassification Report:\n", classification_report(y_class_test, pred_class))

Classification Report:

Accuracy score:
 90.02 %
****************************************

Confusion Matrix:
 [[45395  1353]
 [ 4464  7079]]
****************************************

Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.97      0.94     46748
           1       0.84      0.61      0.71     11543

    accuracy                           0.90     58291
   macro avg       0.88      0.79      0.82     58291
weighted avg       0.90      0.90      0.89     58291



In [106]:
from sklearn.linear_model import LinearRegression
Regression_pipe = Pipeline([
     ('scaler',scaler),
     ('Regressor',LinearRegression()) 
    ])

In [107]:
from sklearn.metrics import mean_squared_error

Regression_pipe.fit(X_train, y_reg_train)

pred_test_lass = Regression_pipe.predict(X_test)

print('R squared test set', round(r2_score(y_reg_test,pred_test_lass)*100, 2))

R squared test set 98.13


In [108]:
pickle.dump(classifier_Pipe, open('Rfc_pipe.pkl','wb'))

In [109]:
pickle.dump(Regression_pipe, open('Regression_pipe.pkl','wb'))